In [ ]:
import os
import skimage.io as skio
import skimage.exposure
DATASET_DIR = '/home/rave/tana-crunch/waves/cropmask/data/'
PLANET_IMAGERY = os.path.join(DATASET_DIR,'planet-za')
WV2_LABELS = os.path.join(DATASET_DIR, 'raw/wv2/gridded_wv2_labels')
planet_img_list = os.listdir(PLANET_IMAGERY)
wv2_labels_list = os.listdir(WV2_LABELS)

### Need to match growing season planet imagery with correct labels and deal with the mismatch in extent due to bufferring. Figure out why buffering was implemented. Apply cloud masking and remove scenes with too much no data that coincides with labels (may need a simpler method of tossing out data just based on amount of no data). Figure out why not exact match between grid lists with Tammy.

In [ ]:
label_ids = sorted([labelid[0:9] for labelid in wv2_labels_list])
img_ids = sorted([imgid[0:9] for imgid in planet_img_list if 'GS' in imgid])

print(len(label_ids))
print(len(img_ids))
list(set(img_ids).intersection(img_ids))


In [ ]:
label_ids

In [ ]:
sorted(img_ids)

In [ ]:
matches = []
for label, img in zip(label_ids, img_ids):
    if label == img:
        matches.append(label)
len(matches)

In [ ]:
matches

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import exposure

def percentile_rescale(arr):
    '''
    Rescales and applies other exposure functions to improve image vis. 
    http://scikit-image.org/docs/dev/api/skimage.exposure.html#skimage.exposure.rescale_intensity
    '''
    rescaled_arr = np.zeros_like(arr)
    for i in range(0,arr.shape[-1]):
        val_range = (np.percentile(arr[:,:,i], 0), np.percentile(arr[:,:,i], 99))
        rescaled_channel = exposure.rescale_intensity(arr[:,:,i], val_range)
        rescaled_arr[:,:,i] = rescaled_channel
        #rescaled_arr= exposure.adjust_gamma(rescaled_arr, gamma=1) #adjust from 1 either way
#     rescaled_arr= exposure.adjust_sigmoid(rescaled_arr, cutoff=.50) #adjust from .5 either way 
    return rescaled_arr

for img_name in planet_img_list[::25]:
    img = skio.imread(img_name)
    r = img[:,:,2]
    g = img[:,:,1]
    b = img[:,:,0]
    img = np.dstack([r/r.max(),g/g.max(),b/b.max()])
    img[img < 0] = 0
    img = percentile_rescale(img)
    plt.figure()
    skio.imshow(img, cmap='brg')

In [ ]:
img.shape